# Quantile Regression 
>__Auteurs__: Yani Bouaffad, Ryma Lakehal, Loic Sauton

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cvx
import matplotlib.pyplot as plt
from download import download
import statsmodels.formula.api as ols
import time
import statsmodels.api as sm
from pathlib import Path
import pandas as pd
from sklearn.linear_model import RidgeCV, Ridge
import ipywidgets as ip
%matplotlib inline
import seaborn as sns
sns.set(context='talk', palette='colorblind', style='darkgrid')


In this part, we were inspired by Joly Julien , Mohamed Sahardid et Anas El Benna's homework to import the dataset "Mesure journalière Région Occitanie Polluants Principaux" and also for camparing what we will foundwith thiers results. 

In [2]:
url = "http://josephsalmon.eu/enseignement/datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
path_target = "datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
download(url, path_target, replace=True)
occ_raw = pd.read_csv(path_target)
occ_raw['date'] = pd.to_datetime(occ_raw['date_debut']).dt.to_period('M')

occ = occ_raw.dropna()
occ.describe()


file_sizes: 100%|██████████████████████████| 5.59M/5.59M [00:02<00:00, 2.03MB/s]
Successfully downloaded file to datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv


C:\Users\rymi-\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1102: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


,X,Y,id,code_insee,id_poll_ue,valeur_originale,x_l93,y_l93,x_wgs84,y_wgs84,FID
count,20808.000000,20808.000000,20808.000000,20808.000000,20808.000000,20808.000000,20808.000000,2.080800e+04,20808.000000,20808.000000,20808.000000
mean,2.294313,43.571370,10404.500000,39870.082612,833.119906,33.823889,642723.199923,6.276022e+06,2.294313,43.571370,10404.500000
std,1.445291,0.317169,6006.896536,17521.409462,2065.511158,28.775214,116870.515281,3.490882e+04,1.445291,0.317169,6006.896536
min,-0.051944,42.684000,1.000000,9225.000000,1.000000,-1.000000,451389.000000,6.176050e+06,-0.051944,42.684000,1.000000
25%,1.418610,43.556900,5202.750000,31069.000000,5.000000,10.100000,572227.000000,6.274400e+06,1.418610,43.556900,5202.750000
50%,1.622590,43.610600,10404.500000,31555.000000,7.000000,20.167708,587812.000000,6.279360e+06,1.622590,43.610600,10404.500000
75%,3.886890,43.695400,15606.250000,34255.000000,7.000000,59.000000,771643.000000,6.288810e+06,3.886890,43.695400,15606.250000
max,4.697360,44.466500,20808.000000,81065.000000,6001.000000,143.498962,836001.000000,6.375360e+06,4.697360,44.466500,20808.000000


In [3]:
occ["nom_com"].value_counts()

TOULOUSE                    5083
LOURDES                     1197
MONTPELLIER                 1190
NIMES                       1113
BLAGNAC                      897
TARBES                       840
CASTRES                      807
LUNEL-VIEL                   801
ALBI                         799
LA CALMETTE                  797
SAINT-GELY-DU-FESC           724
SAINT-LAURENT-DES-ARBRES     671
PERPIGNAN                    629
PEYRUSSE-VIEILLE             543
BESSIERES                    524
AGDE                         472
PAMIERS                      363
SAZE                         361
SAINT-ESTEVE                 360
LATTES                       359
MILLAU                       358
BELESTA-EN-LAURAGAIS         356
CORNEILHAN                   346
GAUDONVILLE                  342
MONTGISCARD                  337
MIRAMONT-DE-COMMINGES        241
FRAISSE-SUR-AGOUT            107
RODEZ                         63
CAHORS                        56
SAINT-GAUDENS                 54
SAINT-PRIV

### liste de ville

In [4]:
ville = 'MONTPELLIER', 'TOULOUSE', 'AGDE'

df = occ[occ['nom_com'].isin(ville)]
df = df[df['polluant'] == 'O3']

df.date.unique()

df = df[df.date == '2018-07']
df = df[['nom_com', 'valeur_originale']]

vecteurs les relevés d'ozone des villes choisies

In [5]:
valeur_o3 = []
len_villes = []

for i in range(len(ville)):

    o3_i = df[df['nom_com'] == ville[i]].values[:, 1]

    # valeurs de O3 des villes choisies
    valeur_o3 = np.append(valeur_o3, o3_i)

    # Nombre d'observations de chaque ville
    len_villes = np.append(len_villes, o3_i.shape)

X = np.zeros((int(np.sum(len_villes)), len(ville)))

for i in range(len(ville)):
    X[int(np.sum(len_villes[0:i])):int(np.sum(len_villes[0:i + 1])
                                       ), i] = np.ones(
                                           int(np.sum(len_villes[i])))

# Quantile regression

Traditionally, the linear regression model for calculating the mean takes the form
$$y_i=\beta_0 + \beta_1 x_{i1} + \dots + \beta_p x_{ip} \ \ i=1, \dots , n$$
where p is equal to the number of features in the equation and n is the number of data points. The best linear regression line is found by minimizing the mean square error, which is found with the equation
$$ 
$$

blabla je finirai plus tard 

## implementing quantile regression using CVXPY package

CVXPY is a Python-embedded modeling language for convex optimization problems. It allows us to express our problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

In [6]:
alphas = np.linspace(0.0, 1, 11)

In [16]:
Y = valeur_o3
X = X

fit = cvx.Variable(len(X[1]))
alpha = cvx.Parameter(value=0.5)

for i in range(len(X)):
    f = Y[i] - (fit.T * X[i])

error = cvx.sum(0.5 * cvx.abs(f) + (alpha - 0.5) * f)

objective = cvx.Minimize(error)
problem = cvx.Problem(objective)

C:\Users\rymi-\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\rymi-\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\rymi-\anaconda3\lib\site-packages\cvxpy\expressions\express

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1